# RNNを用いた言語モデル

RNNを用いて言語モデルを実装する。

前章では、深層学習を活用してある単語から次の単語を予測するモデルを作成した。しかし、これでは全体の文脈全体を考慮できない。具体的には、2つ以上前の単語を考慮した予測が出来ない。

RNNを用いることで、文脈を考慮した予測が可能になる。  
本章ではRNNの構造について学び、RNNを用いた言語モデルを実装する。

In [1]:
import random

import sentencepiece as spm
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence
from dlprog import train_progress

In [2]:
prog = train_progress(width=20, with_test=True, label="ppl train", round=2)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

### 学習データの用意

In [3]:
n_data = 10000

wiki40b

In [4]:
text_path = "data/jawiki.txt"
with open(text_path) as f:
    data = f.read().splitlines()

In [5]:
data = data[:n_data]
text_path = f"data/jawiki_{n_data}.txt"
with open(text_path, 'w') as f:
    f.write("\n".join(data))

print("num of data:", len(data))
data[:3] # examples

num of data: 10000


['「教科書には決して載らない」日本人の謎やしきたりを多角的に検証し、日本人のDNAを解明する。新春番組として定期的に放送されており、年末の午前中に再放送されるのが恒例となっている。',
 'ライブドア社員であった初代代表取締役社長の山名真由によって企業内起業の形で創業。2005年に株式会社ライブドアから分割されて設立。かつてはライブドアホールディングス（現・LDH）の子会社であったが、ノンコア事業の整理にともない、株式会社ゲオ（現：株式会社ゲオホールディングス）に所有する全株式を譲渡し、同社の完全子会社となった。「ぽすれん」「ゲオ宅配レンタル」のオンラインDVD・CD・コミックレンタルサービス及び「GEO Online」と「ゲオアプリ」のアプリ・ウェブサイト運営の大きく分けて2事業を展開している。以前はDVD販売等のEコマースサービス「ぽすれんストア」、動画配信コンテンツ「ぽすれんBB」や電子書籍配信サービスの「GEO☆Books」事業も行っていた。オンラインDVDレンタル事業では会員数は10万人（2005年9月時点）。2006年5月よりCDレンタルを開始。同業他社には、カルチュア・コンビニエンス・クラブが運営する『TSUTAYA DISCAS』のほか、DMM.comが運営する『DMM.com オンラインDVDレンタル』がある。過去には「Yahoo!レンタルDVD」と「楽天レンタル」の運営を受託していた。',
 '2005年の一時期、東京のラジオ局、InterFMで、「堀江社長も使っているライブドアのぽすれん」というキャッチコピーでラジオCMを頻繁に行っていたことがあった。']


---

## RNN

*Recurrent Neural Network*

再帰型ニューラルネットワーク。  
再帰的な構造を持つニューラルネットワークで、可変長の時系列データを扱うことが得意。

RNNはある時間$t$の入力$x_t$に対して以下のような演算を行い、出力値$h_t$を決定する。

$$
h_t = \text{tanh}(W_x x_t + b_x + W_h h_{t-1} + b_h)
$$

演算の内部で前の時間の出力値$h_{t-1}$を参照していることが分かる。  
演算内容はシンプルに捉えることができて、入力$x_t$と前の時間の出力$h_{t-1}$をそれぞれ線形変換し、それらの和を活性化関数$\text{tanh}$に通しているだけ。線形変換に必要な重みとバイアスが2つずつあるので、パラメータは合計4つ。  
活性化関数は$\text{tanh}$でなくてもよいが、大体は$\text{tanh}$が使われる。

実装してみるとこんな感じ。

In [6]:
class RNNCell(nn.Module):
    def __init__(self, input_size: int, output_size: int):
        super().__init__()
        self.fc_input = nn.Linear(input_size, output_size)
        self.fc_output = nn.Linear(output_size, output_size)

    def forward(self, x, h):
        """
            x: (batch_size, input_size)
            h: (batch_size, output_size)
        """
        z = self.fc_input(x) + self.fc_output(h) # (batch_size, output_size)
        h = F.tanh(z)
        return h

推論の流れを見てみる。  
まず適当なパラメータで出力の初期値$h_0$と初めの入力$x_1$を定義する。$h_0$は0ベクトルで良い。

In [7]:
batch_size, input_size, hidden_size = 2, 3, 4
rnn = RNNCell(input_size, hidden_size)

x1 = torch.randn(batch_size, input_size)
h0 = torch.zeros(batch_size, hidden_size)

h1 = rnn(x1, h0)
h1

tensor([[-0.1888,  0.3809, -0.3648,  0.3572],
        [-0.9422, -0.3746, -0.8314,  0.8185]], grad_fn=<TanhBackward0>)

この$h_1$を参照して、次の時間の入力$x_2$に対する出力$h_2$を決定する。

In [8]:
x2 = torch.randn(batch_size, input_size)
h2 = rnn(x2, h1)
h2

tensor([[ 0.6576,  0.8093, -0.6598,  0.6559],
        [ 0.0888,  0.8746, -0.3826, -0.3870]], grad_fn=<TanhBackward0>)

これがRNNの推論の流れである。  
この一連の流れをまとめたモデルを実装する。

In [9]:
class RNN(nn.Module):
    def __init__(self, input_size: int, hidden_size: int):
        super().__init__()
        self.rnn_cell = RNNCell(input_size, hidden_size)

    def forward(self, x, h):
        """
            x: (seq_len, batch_size, input_size)
            h: (batch_size, hidden_size)
        """
        hs = []
        for xi in x:
            h = self.rnn_cell(xi, h)
            hs.append(h)
        hs = torch.stack(hs) # (seq_len, batch_size, hidden_size)
        return hs

任意の長さの入力に対して、同じ数の隠れ状態を出力する。

In [10]:
batch_size, input_size, hidden_size, seq_len = 2, 3, 4, 5
rnn = RNN(input_size, hidden_size)
x = torch.randn(seq_len, batch_size, input_size)
h = torch.zeros(batch_size, hidden_size)

hs = rnn(x, h)
hs.shape

torch.Size([5, 2, 4])

少し扱いやすくしたいので書き換える。以下を実現する。
- 系列長の次元ではなくバッチ次元を初めにした形状`(batch_size, seq_len, hidden_size)`で入出力を扱えるようにする。
- 隠れ状態`h`の入力がなかった場合に自動で0ベクトルになるようにする。

In [11]:
class RNN(nn.Module):
    def __init__(self, input_size: int, hidden_size: int):
        super().__init__()
        self.rnn_cell = RNNCell(input_size, hidden_size)
        self.hidden_size = hidden_size

    def forward(self, x, h=None):
        """
            x: (batch_size, seq_len, input_size)
            h: (batch_size, hidden_size)
        """
        if h is None:
            h = torch.zeros(x.size(0), self.hidden_size).to(x.device)
        hs = []
        x = x.transpose(0, 1) # (seq_len, batch_size, input_size)
        for xi in x:
            h = self.rnn_cell(xi, h)
            hs.append(h)
        hs = torch.stack(hs) # (seq_len, batch_size, hidden_size)
        hs = hs.transpose(0, 1) # (batch_size, seq_len, hidden_size)
        return hs

`transpose(0, 1)`でバッチ次元と系列長の次元を入れ替える事で実現した。

In [12]:
batch_size, input_size, hidden_size, seq_len = 2, 3, 4, 5
rnn = RNN(input_size, hidden_size)
x = torch.randn(batch_size, seq_len, input_size)
h = torch.zeros(batch_size, hidden_size)

hs = rnn(x, h)
hs.shape

torch.Size([2, 5, 4])

### PyTorchでの実装

PyTorchにはRNNを実装するためのクラスが用意されている。軽く紹介する。

#### `torch.nn.RNNCell`

RNNの一つの時間の演算を行うクラス。初めに実装したもの。  
https://pytorch.org/docs/stable/generated/torch.nn.RNNCell.html

In [13]:
batch_size, input_size, hidden_size = 2, 3, 4

rnn_cell = nn.RNNCell(input_size, hidden_size)
for params in rnn.parameters():
    print(params, "\n")

Parameter containing:
tensor([[ 0.2187, -0.5637, -0.0150],
        [-0.3830,  0.1954,  0.2820],
        [-0.4701, -0.3419, -0.0627],
        [-0.4548, -0.4809,  0.2266]], requires_grad=True) 

Parameter containing:
tensor([-0.5662,  0.1319,  0.2761, -0.1996], requires_grad=True) 

Parameter containing:
tensor([[ 0.0360, -0.1851, -0.4481, -0.1813],
        [-0.3415,  0.3094, -0.0361,  0.1395],
        [ 0.0090,  0.4786,  0.4440,  0.3626],
        [ 0.2033, -0.1460, -0.3531, -0.2259]], requires_grad=True) 

Parameter containing:
tensor([ 0.1171,  0.0946, -0.3413, -0.3185], requires_grad=True) 



ちゃんとパラメータが4つあるね。

推論も同じ。

In [14]:
x = torch.randn(batch_size, input_size)
h = torch.zeros(batch_size, hidden_size)
rnn_cell(x, h)

tensor([[-0.7446, -0.8601,  0.0171,  0.1176],
        [-0.7355, -0.8861, -0.1506,  0.0908]], grad_fn=<TanhBackward0>)

ちなみに$h$は入力しなくてもいい。その場合は勝手に0ベクトルになる。

In [15]:
rnn_cell(x)

tensor([[-0.7446, -0.8601,  0.0171,  0.1176],
        [-0.7355, -0.8861, -0.1506,  0.0908]], grad_fn=<TanhBackward0>)

#### `torch.nn.RNN`

全ての時間の演算をまとめて行うクラス。  
https://pytorch.org/docs/stable/generated/torch.nn.RNN.html

In [16]:
batch_size, input_size, hidden_size, seq_len = 2, 3, 4, 5
rnn = nn.RNN(input_size, hidden_size, batch_first=True)

x = torch.randn(batch_size, seq_len, input_size)
hs, h = rnn(x)
hs.shape

torch.Size([2, 5, 4])

全ての時間の隠れ状態が出力される。`batch_first=True`とすることでバッチ次元を初めできる。

最後の時間の隠れ状態も別で出力される。

In [17]:
hs[:, -1] == h

tensor([[[True, True, True, True],
         [True, True, True, True]]])

In [18]:
h.shape

torch.Size([1, 2, 4])

初めに次元が1つ追加されるのは仕様。


---

## RNNを用いた言語モデル

RNNを用いた言語モデル構築の流れを見てみる。

基本的に、RNNそのものを1つのモデルとして扱うことはない。RNNはNNの中の1つの層として扱う。  
なお、RNNが組み込まれたモデルをRNNと呼ぶこともある。混乱を避けるために、本章では層としてのRNNはRNN層と呼ぶこととする（モデルとしてのRNNはそのままRNNと呼ぶ）。

ある時間$t$におけるRNN層は、その時間の入力$x_t$と前の時間のRNN層の出力$h_{t-1}$を受け取り、$h_t$を出力する。  
$h_t$は外から直接観測されることがないため、**隠れ状態**（や潜在変数）と表現される。変数名に$h$を使っている理由がそこにある（隠れる=hide）。

では、RNN層をNNに取り入れて言語モデルを作ろう。以下のような最低限のネットワークを構築する。

In [19]:
class LanguageModel(nn.Module):
    def __init__(self, n_vocab, embed_dim, hidden_size):
        super().__init__()
        self.embedding = nn.Embedding(n_vocab, embed_dim)
        self.rnn = RNN(embed_dim, hidden_size)
        self.fc = nn.Linear(hidden_size, n_vocab)

    def forward(self, x, h=None):
        """
        x: (batch_size, seq_length)
        h: (1, batch_size, hidden_size)
        """
        x = self.embedding(x) # (batch_size, seq_length, embed_dim)
        hs = self.rnn(x, h) # hs: (batch_size, seq_length, hidden_size)
        y = self.fc(hs) # (batch_size, seq_length, n_vocab)
        return y, hs

このモデルで再帰的な推論を行うことで文章生成が可能になる。初めの単語$w_0$（`bos`）と隠れ状態$h_0$（0ベクトル）を入力し、次の単語$w_1$と隠れ状態$h_1$を得る。次に$w_1$と$h_1$を再度モデルに入力し、$w_2$と$h_2$を得る。これを繰り返すことで、文脈を考慮した文章生成を実現する（この詳細は後のBPTTの章にて説明する）。

流れを見てみよう。

In [20]:
n_vocab, embed_dim, hidden_size = 8000, 512, 512
model = LanguageModel(n_vocab, embed_dim, hidden_size)

テストなので、語彙数は少なめに10としている。  
次に初めの単語$w_0$を用意する。

In [21]:
w0 = torch.tensor(0).reshape(1, -1) # (batch_size, seq_length) = (1, 1)
w0

tensor([[0]])

初めの隠れ状態$h_0$も用意し、これらをモデルに入力する。

In [22]:
h0 = torch.zeros(1, hidden_size)
y1, h1 = model(w0, h0)
y1.shape, h1.shape

(torch.Size([1, 1, 8000]), torch.Size([1, 1, 512]))

出力を確率分布に変換し、次の単語をサンプリングする。

In [23]:
dist = F.softmax(y1.squeeze(), dim=-1)
w1, = random.choices(range(n_vocab), weights=dist)
w1

355

これを繰り返すことで文章を生成する。


---

## 学習データの作成

モデルの学習に使用する、入力と出力のペアを作成する。どんなペアを作成すれば良いだろうか。

欲しいモデルは、可変長の単語列から次の単語を予測するモデルである。これを考えると、ある時間$t$までの単語列を入力、$t+1$の単語を正解とするペアを作成すれば良さそう。

入力 | 正解
--- | ---
私 | は
私 は | 今日
私 は 今日 | オムライス
私 は 今日 オムライス | を
$\vdots$ | $\vdots$

みたいな感じ。

これでもいいが、もう少しRNNの力を活かす方法がある。  
RNNは各時間で予測単語を出力する。例えば、「私 は 今日」という3つの単語を入力した時、RNNLMは「私」の次に来る単語、「私 は」の次に来る単語、「私 は 今日」の次に来る単語、という3つの単語を1度に出力する。この3つの単語の誤差とその勾配は一度に求められる。

ということで、入力と正解のペアは以下のような形で文章ごとに用意すればいい。

入力 | 正解
--- | ---
私 は 今日 オムライス を 食べ | は 今日 オムライス を 食べ た
昨日 は 大雨 だっ | は 大雨 だっ た
YOASOBI の ボーカル が かわい | の ボーカル が かわい い
AI が 人間 の 仕事 を 奪 | が 人間 の 仕事 を 奪 う
$\vdots$ | $\vdots$

こんな感じに、単語を1つずらしたものが正解になるね。こうすれば文脈と正解の組み合わせが全て網羅できる。

In [24]:
class TextDataset(Dataset):
    def __init__(self, data_ids):
        self.data = [torch.tensor(ids) for ids in data_ids]
        self.n_data = len(data_ids)

    def __getitem__(self, idx):
        text = self.data[idx]
        in_text = text[:-1] # 入力単語列
        out_text = text[1:] # 入力を1つずつずらした単語列
        return in_text, out_text

    def __len__(self):
        return self.n_data


---

## BPTT

*Back Propagation Through Time*

時間を跨いだ逆伝播。

RNNでは、ある時間$t$における演算で、その時間の入力$x_t$以外に、隠れ状態（前のRNN層からの出力）$h_{t-1}$を参照する。  
これは$x_t$のみを参照する通常のNNとは大きく異なる点である。

言語モデルの様な、各時間で確率分布からのサンプリングを挟むようなモデルでは、この隠れ状態が重要な役割を果たす。それは逆伝播が時間を跨いで繋がる点である。これをBPTT (*Back Propagation Through Time*) と呼ぶ。

BPTTによって時間を遡って勾配を届けられるようになり、長期的な視点で誤差が減るように学習できる。その際、勾配の伝達を担う隠れ状態には、長期的な文脈の情報が載るようになることが期待される。

ちなみに、サンプリングを挟まない場合は普通に逆伝播が届く。モデルの出力値をそのまま入力する場合とか。  
あとは、サンプリングといっても、出力されたデータの一部を次に入力するような場合も普通に逆伝播が届く。

### Truncated BPTT

BPTTで細かく時間を区切る手法。

BPTTには一つ問題があり、それは多くのメモリを要することである。系列長が長くなればなるほど多くのメモリが必要になる。

これを解決する方法として、Truncated BPTTというものがある。これは、逆伝播の際に勾配の流れを一定の長さで区切る手法である。これによってメモリの消費を抑える。  
隠れ状態の流れを切ることとなり、長期的な文脈を考慮するための勾配が届かなくなる。ただ、実はRNNの時点で長期的な文脈の考慮は難しいため、大きな影響にはならない。

実装は以下のようになる。

```python
trunc_len = 64 # 区切る長さ

for x, t in dataloader:
    h = None # 隠れ状態の初期化

    # Truncated BPTT
    for i in range(0, x.shape[1], trunc_len):
        x_batch = x[:, i:i+trunc_len].to(device)
        t_batch = t[:, i:i+trunc_len].to(device)

        optimizer.zero_grad()
        y, h = model(x_batch, h)
        loss = loss_fn(y, t_batch)
        loss.backward()
        optimizer.step()

        h = h.detach() # 計算グラフから切り離す
```

あくまでも区切るのは逆伝播だけなので、順伝播時の隠れ状態はちゃんと保持される。


---

## ミニバッチ学習

RNNを用いた言語モデルの学習でミニバッチ学習を行うには少し工夫がいる。というのも、文章ごとに長さが違うため、普通にやってもミニバッチ内でデータのサイズが異なってしまう。  
そこで、パディングという操作を行い、バッチ内のデータの長さを揃える。パディング用の特殊トークンを用意し、バッチ内の一番長いデータに合わせてパディングする。具体的には、足りない長さをパディング用のトークンで埋める。

こんな感じ。

In [25]:
from torch.nn.utils.rnn import pad_sequence

In [26]:
sample = [
    torch.tensor([1, 2, 3]),
    torch.tensor([1, 2]),
    torch.tensor([1, 2, 3, 4, 5]),
]
pad_sequence(sample, batch_first=True, padding_value=0) # 0でパディング

tensor([[1, 2, 3, 0, 0],
        [1, 2, 0, 0, 0],
        [1, 2, 3, 4, 5]])

パディング用のトークンidとして0を設定し、最大の長さ5に満たないデータに対して0を埋めて長さを揃えた。

これを用いて学習データを作成する。  
まずpadトークンを入れたトークナイザを作る。

In [ ]:
pad_id = 3
vocab_size = 8000
tokenizer_prefix = f"models/tokenizer_jawiki_{n_data}"

spm.SentencePieceTrainer.Train(
    input=text_path,
    model_prefix=tokenizer_prefix,
    vocab_size=vocab_size,
    pad_id=pad_id, # padトークンのIDを指定
)

これでトークン化。

In [28]:
sp = spm.SentencePieceProcessor(f"{tokenizer_prefix}.model")
n_vocab = len(sp)

unk_id = sp.unk_id()
bos_id = sp.bos_id()
eos_id = sp.eos_id()

data_ids = sp.encode(data)
for ids in data_ids:
    ids.insert(0, bos_id)
    ids.append(eos_id)

n_vocab = len(sp)
print("num of vocabrary:", n_vocab)
data_ids[0][:10] # example

num of vocabrary: 8000


[1, 11, 20, 501, 557, 366, 51, 658, 58, 3734]

そんで、バッチ内データが揃う用にDataLoaderを作成する。

In [29]:
def collate_fn(batch):
    """ミニバッチ内のデータをパディングによって揃える"""
    in_text, out_text = zip(*batch)
    in_text = pad_sequence(in_text, batch_first=True, padding_value=pad_id)
    out_text = pad_sequence(out_text, batch_first=True, padding_value=pad_id)
    return in_text, out_text

batch_size = 64
dataset = TextDataset(data_ids)
train_dataset, test_dataset = random_split(dataset, [0.8, 0.2])

train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    collate_fn=collate_fn # 取得したミニバッチに対して行う処理の指定
)
test_loader = DataLoader(
    test_dataset,
    batch_size=batch_size,
    collate_fn=collate_fn
)

sample_x, sample_y = next(iter(train_loader))
print("input batch shape", sample_x.shape)
sample_x[0][:10], sample_y[0][:10] # example

input batch shape torch.Size([64, 957])


(tensor([   1,   11, 5426,   12,   38,   22,  244,   34, 3977, 3329]),
 tensor([  11, 5426,   12,   38,   22,  244,   34, 3977, 3329,    5]))

これで学習データはOK。

最後に、学習時に使う交差エントロピーでパディング用のトークンを無視するように定義すれば完了。

In [30]:
cross_entropy = nn.CrossEntropyLoss(ignore_index=pad_id)


---

## 実践

実際にモデルを作って文章を生成してみよう。

先程作成したモデルにいくつかの層を追加する。

In [31]:
class LanguageModel(nn.Module):
    def __init__(self, n_vocab, embed_dim, hidden_size):
        super().__init__()
        self.embedding = nn.Embedding(n_vocab, embed_dim)
        self.rnn = RNN(embed_dim, hidden_size)
        self.fc_hidden = nn.Linear(hidden_size, hidden_size)
        self.norm = nn.LayerNorm(hidden_size)
        self.relu = nn.ReLU()
        self.fc_out = nn.Linear(hidden_size, n_vocab)
        self.dropout = nn.Dropout(0.2)
        self.fc_skip = nn.Linear(embed_dim, hidden_size)

    def forward(self, x, h=None):
        """
        x: (batch_size, seq_length)
        h: (1, batch_size, hidden_size)
        """
        x = self.embedding(x) # (batch_size, seq_length, embed_dim)
        x = self.dropout(x)
        skip = self.fc_skip(x) # (batch_size, seq_length, hidden_size)

        hs = self.rnn(x, h) # hs: (batch_size, seq_length, hidden_size)
        hs = hs + skip
        hs = self.dropout(hs)

        skip = hs
        hs = self.fc_hidden(hs) # (batch_size, seq_length, hidden_size)
        hs = self.norm(hs)
        hs = self.relu(hs)

        hs = hs + skip
        y = self.fc_out(hs) # (batch_size, seq_length, n_vocab)
        return y, hs

線形層（+活性化関数）、正規化層、ドロップアウト層、残差結合を追加した。

In [32]:
n_vocab = len(sp)
embed_dim = 512
hidden_size = 512
model = LanguageModel(n_vocab, embed_dim, hidden_size).to(device)
n_params = sum(p.numel() for p in model.parameters())
print(f"num of parameters: {n_params:,}")

num of parameters: 9,251,648


### 学習

まずは学習。

In [33]:
def loss_fn(y, t):
    """
    y: (batch_size, seq_length, n_vocab)
    t: (batch_size, seq_length)
    """
    loss = cross_entropy(y.reshape(-1, n_vocab), t.ravel())
    return loss


def eval_model(model, trunc_len=100):
    model.eval()
    losses = []
    with torch.no_grad():
        for x, t in test_loader:
            h = None
            for i in range(0, x.shape[1], trunc_len):
                x_batch = x[:, i:i+trunc_len].to(device)
                t_batch = t[:, i:i+trunc_len].to(device)
                y, hs = model(x_batch, h)
                loss = loss_fn(y, t_batch)
                losses.append(loss.item())
                h = hs[:, -1]
    loss = sum(losses) / len(losses)
    ppl = torch.exp(torch.tensor(loss)).item()
    return ppl


def train(model, optimizer, trunc_len, n_epochs, prog_unit=1):
    # trunc_len: Truncated BPTTで区切る長さ

    prog.start(n_iter=len(train_loader), n_epochs=n_epochs, unit=prog_unit)
    for _ in range(n_epochs):
        model.train()
        for x, t in train_loader:
            h = None

            # Truncated BPTT
            for i in range(0, x.shape[1], trunc_len):
                trunc_x = x[:, i:i+trunc_len].to(device)
                    # (batch_size, trunc_len, n_vocab)
                trunc_t = t[:, i:i+trunc_len].to(device)
                    # (batch_size, trunc_len)

                optimizer.zero_grad()
                y, hs = model(trunc_x, h)
                loss = loss_fn(y, trunc_t)
                loss.backward()
                optimizer.step()
                ppl = torch.exp(loss).item()
                prog.update(ppl, advance=0)
                hs = hs.detach() # 計算グラフから切り離す
                h = hs[:, -1] # 最後の隠れ状態
            prog.update()

        if prog.now_epoch % prog_unit == 0:
            test_ppl = eval_model(model)
            prog.memo(f"test: {test_ppl:.5f}", no_step=True)
        prog.memo()

In [34]:
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [35]:
train(model, optimizer, trunc_len=100, n_epochs=100, prog_unit=5)

    1-5/100: #################### 100% [00:07:22.78] ppl train: 594.04, test: 344.12152 
   6-10/100: #################### 100% [00:07:19.77] ppl train: 222.52, test: 285.51825 
  11-15/100: #################### 100% [00:07:16.97] ppl train: 167.26, test: 261.12546 
  16-20/100: #################### 100% [00:07:27.67] ppl train: 139.84, test: 249.88216 
  21-25/100: #################### 100% [00:07:17.98] ppl train: 124.95, test: 245.01172 
  26-30/100: #################### 100% [00:07:26.28] ppl train: 113.35, test: 239.42883 
  31-35/100: #################### 100% [00:07:15.80] ppl train: 105.92, test: 242.02299 
  36-40/100: #################### 100% [00:07:18.20] ppl train: 99.70, test: 241.37332 
  41-45/100: #################### 100% [00:07:16.60] ppl train: 95.81, test: 244.94292 
  46-50/100: #################### 100% [00:07:15.60] ppl train: 92.09, test: 243.68498 
  51-55/100: #################### 100% [00:07:21.67] ppl train: 88.63, test: 246.44403 
  56-60/100: ############

In [36]:
model_path = "models/lm_rnn.pth"
torch.save(model.state_dict(), model_path)

### 文章生成

学習したモデルを使って文章を生成する。  
モデルが出力した単語を次の入力とし、再帰的に出力を得る。

In [37]:
model = LanguageModel(n_vocab, embed_dim, hidden_size).to(device)
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

In [38]:
def token_sampling(y):
    y = y.squeeze()
    y[unk_id] = -torch.inf
    probs = F.softmax(y, dim=-1)
    token, = random.choices(range(n_vocab), weights=probs)
    return token

@torch.no_grad()
def generate_sentence(
    model: nn.Module,
    start: str = '',
    max_len: int = 50
) -> str:
    model.eval()

    # 隠れ状態の初期値
    token_ids = sp.encode(start)
    token_ids.insert(0, bos_id)
    x = torch.tensor(token_ids, device=device).unsqueeze(0)
    y, hs = model(x)
    h = hs[:, -1]
    next_token = token_sampling(y)
    token_ids.append(next_token)

    # 続きの文章生成
    while len(token_ids) <= max_len and next_token != eos_id:
        x = torch.tensor([next_token], device=device).unsqueeze(0)
        y, (h,) = model(x, h)
        next_token = token_sampling(y)
        token_ids.append(next_token)

    sentence = sp.decode(token_ids)
    return sentence

In [40]:
for _ in range(5):
    print(generate_sentence(model, max_len=50))

大道者個人により、「国内では、「鈴木道曜深夜の前から、「『ある革命なの)は、し、&返し、ブの元マイパー 元帥ゲーリングに就任。4165月5月4日
Q以外も同様Tix芸人というNHKが存在する。
ベトナムジャ記録
日本では思いではウェストスとローズがH.6時間が頻、「Myで知れ、ゲーリングは1時間(ユダヤ人の大隊は公知のではなく、ゲーリングがゲーリングがゲーリングだった」とゲーリングはクラスの際にはゲーリング
ホイヤーズアルジェ・。


マルコフモデルとは違い、文脈全体を考慮した予測がされているため、文法の崩れや文章全体での不自然さが減ったと思われる。